<a href="https://colab.research.google.com/github/D-Shah28/Prasunet_DS_03/blob/main/Prasunet_DS_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decision Tree Classifier

To predict whether a customer will purchase a product or service based on their demographic and behavioral data.

Dataset used ->>Bank Marketing dataset from the UCI Machine Learning Repository.

In [81]:
# importing the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import export_graphviz

Data collection

In [54]:
#loading data from the github repository to pandas dataframe
bank_df = pd.read_csv('https://raw.githubusercontent.com/D-Shah28/Prasunet_DS_03/main/bank-additional-full.csv')

In [55]:
#displaying first 5 column of dtaset to understand it
bank_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Data PreProcessing

In [56]:
# check the sahpe of the dataset
bank_df.shape

(41188, 21)

In [57]:
#check the missing value of the dataset
bank_df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

as there are no missing value we dont have to do any cleaning in the data

In [58]:
# check the datatypes of column
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [59]:
#check the distribution of target column (y)
bank_df['y'].value_counts()

y
no     36548
yes     4640
Name: count, dtype: int64

as there is a significant difference in the distribution of the target column which can lead to a baised model so first we need to make the distribution similar by using Undersampling method

In [62]:
# seperating the data as per the target variable ('y) -> into Yes or No variables
Yes = bank_df[bank_df.y == "yes"]
No = bank_df[bank_df.y == "no"]

In [63]:
print(Yes.shape)
print(No.shape)

(4640, 21)
(36548, 21)


In [64]:
# using undersampling method to reduce the no data to similar to yes data in the dataset
No_sample = No.sample(n=4640)

In [65]:
# check the shape of reduced data distribution
print(No_sample.shape)

(4640, 21)


In [68]:
#concatenating the whole data as a new_bank_df dataset
new_bank_df = pd.concat([Yes, No_sample], axis=0)

In [69]:
#check the shape of new dataset
new_bank_df.shape

(9280, 21)

In [71]:
# converting the categorical variables to numerical variables
# by using LabelEncoder
non_numerical_cols = new_bank_df.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in non_numerical_cols:
   new_bank_df[col] = le.fit_transform(new_bank_df[col])

In [73]:
#  check the updated dataset
new_bank_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
75,41,1,0,0,1,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,1
83,49,2,1,6,1,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,1
88,49,9,1,2,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,1
129,41,9,1,5,1,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,1
139,45,1,1,2,1,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,1


In [87]:
# seperating the features and target variable from the datase
X = new_bank_df.drop(columns='y',axis=1)
Y = new_bank_df['y']

In [88]:
print(X)
print(Y)

       age  job  marital  education  default  housing  loan  contact  month  \
75      41    1        0          0        1        2     0        1      6   
83      49    2        1          6        1        2     0        1      6   
88      49    9        1          2        0        0     0        1      6   
129     41    9        1          5        1        2     0        1      6   
139     45    1        1          2        1        2     0        1      6   
...    ...  ...      ...        ...      ...      ...   ...      ...    ...   
29762   46    7        1          3        1        2     0        0      0   
18239   56    5        0          3        0        2     0        0      3   
29380   36    1        1          1        0        0     0        0      0   
28208   40    9        1          5        0        2     0        0      0   
5781    38    1        1          1        1        0     0        1      6   

       day_of_week  duration  campaign  pdays  prev

In [89]:
#splitting the dataset into train and test dataset
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)

In [90]:
#check the split of data
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, Y_train.shape, Y_test.shape)

(9280, 20) (7424, 20) (1856, 20)
(9280,) (7424,) (1856,)


Building the Decision Tree Classifier

In [91]:
# loading the decision tree classifier fucntion
classifier = DecisionTreeClassifier()

In [92]:
# fitting the model as per the function
classifier.fit(X_train, Y_train)

DecisionTreeClassifier()

In [93]:
# make predictions on testing dataset
Y_pred = classifier.predict(X_test)

In [94]:
gi# evaluating the model
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:", classification_report(Y_test, Y_pred))
print("Confusion Matrix:", confusion_matrix(Y_test, Y_pred))

Accuracy: 0.8259698275862069
Classification Report:               precision    recall  f1-score   support

           0       0.80      0.85      0.82       889
           1       0.85      0.80      0.83       967

    accuracy                           0.83      1856
   macro avg       0.83      0.83      0.83      1856
weighted avg       0.83      0.83      0.83      1856

Confusion Matrix: [[756 133]
 [190 777]]
